# Wnioski



Dla tabeli name_basics możemy zauważyć, że mamy duży brakujących informacji (\N) o roku urodzenia oraz śmierci danej osoby (chociaż to może wynikać z faktu, że dana osoba wciąż żyje). W naszych danych przeważają aktorzy obu płci. Widoczne jest również, że jedna osoba może mieć wiele profesji lub nie mieć ich wcale (21,6% brakujących danych). 

Z tabeli title_akas możemy zauważyć dużą ilość pseudonulli (ponad 84%) dla kolumny types. Podobnie kolumna attributtes jest złożona z prawie samych pseudonulli - innych danych jest tylko 0,7%. Ponadto ponad 82% produkcji pochodzi z 7 regionów, pozostałe niecałe 18% z reszty świata.

Tabela title_basics pokazuje, że znaczącą przewagę w danych mają tvEpisode (ponad 75%), ponieważ mamy rozbicie na kolejne numery epizodów.  Tylko nieco ponad 3% to treści przeznaczone dla dorosłych. Dodatkowo mamy duże braki (\N)  dla endYear - to z kolei wynika z faktu, że dla filmów ten parametr nie jest podawany. Kolejną wybrakowaną kolumną jest runtimeMinutes - ponad 70% wartości \N. W tabeli przeważają dramaty, choć ponownie, dany film może mieć więcej niż jeden gatunek.

Dla tabeli title_crew w kolumnach directors oraz writers ponownie pojawia się problem braków (ponad 40%). 

Tabela title_episode pokazuje, że połowę naszych danych dotyczących seriali stanowią odcinki pochodzące z pierwszych sezonów. Jednak prawie 21% ma brakującą daną dotyczącą numeru sezonu oraz odcinka.

Z tabeli title_principals możemy ponownie wskazać dominację aktorów oraz aktorek, jednak warto zauważyć, że ponad 17% grało siebie. Dla kolumny job mamy ponad 83% pseudonulli. 

Tabela title_ratings pokazuje, że jest ogromna różnica między minimalną a maksymalną liczbą ocen filmu - minumum z próbki to 5, a maksimum 2038650. Nie jest widoczna korelacją pomiędzy liczbą ocen dla filmu a jego oceną.

# Spark Setup and Data Load

##Installation of Spark

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# unzip it
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
# install findspark
!pip install -q findspark
# Google Colab has Java 11 available, test it using below command -
!ls /usr/lib/jvm
#install pyarrow
!pip install -U pyarrow

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.0 MB 346 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.1 which is incompatible.
db-dtypes 1.0.4 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.1 which is incompatible.


In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark_params = {
"spark.executor.memory" : "4g",
"spark.driver.memory": "4g",
"spark.memory.fraction": "0.9"}
for param, value in spark_params.items():
  spark.conf.set(param, value)

## Load dataset

In [ ]:
!wget "https://datasets.imdbws.com/name.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.akas.tsv.gz"
!wget "https://datasets.imdbws.com/title.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.crew.tsv.gz"
!wget "https://datasets.imdbws.com/title.episode.tsv.gz"
!wget "https://datasets.imdbws.com/title.principals.tsv.gz"
!wget "https://datasets.imdbws.com/title.ratings.tsv.gz"

--2022-11-23 19:58:41--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.40, 13.224.2.111, 13.224.2.93, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236745635 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.78M   138MB/s    in 1.6s    

2022-11-23 19:58:42 (138 MB/s) - ‘name.basics.tsv.gz’ saved [236745635/236745635]

--2022-11-23 19:58:42--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.40, 13.224.2.111, 13.224.2.93, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288222028 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.87M   127MB/s    

In [ ]:
title_ratings = spark.read.csv("title.ratings.tsv.gz", sep='\t', header=True)
title_principals = spark.read.csv("title.principals.tsv.gz", sep='\t',header=True)
title_episode = spark.read.csv("title.episode.tsv.gz", sep='\t', header=True)
title_crew = spark.read.csv("title.crew.tsv.gz", sep='\t', header=True)
title_basics = spark.read.csv("title.basics.tsv.gz", sep='\t', header=True)
title_akas = spark.read.csv("title.akas.tsv.gz", sep='\t', header=True)
name_basics = spark.read.csv("name.basics.tsv.gz", sep='\t', header=True)

## Profilowanie danych

Profilowanie danych będziemy przeprowadzać dla próbek ze wszystkich tabel. Wielkość próbki będzie uzależniona od rozmiaru tabeli.

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     | 22.0 MB 157 kB/s
     |████████████████████████████████| 102 kB 7.5 MB/s 
     |████████████████████████████████| 690 kB 50.7 MB/s 
     |████████████████████████████████| 62 kB 59 kB/s 
     |████████████████████████████████| 9.9 MB 48.4 MB/s 
     |████████████████████████████████| 4.7 MB 47.2 MB/s 
     |████████████████████████████████| 296 kB 54.3 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-dev-py2.py3-none-any.whl size=325000 sha256=dd43ee4dcb8e2a11755f2cd8af8c3998ac93086ae5edb41eeb9bd95f697fbd1a
  Stored in directory: /tmp/pip-ephem-wheel-cache-0wl55516/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=c0f962bc1c753db19e0ef8f48ef76e754b23070e973d90eca0909c2e3038aaeb
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c32

In [ ]:
def profile(data, sample_size, report_name):
  sample = data.sample(sample_size)
  data_pd = sample.toPandas()
  profile = ProfileReport(data_pd, title = f"Pandas Profiling Report {report_name}")
  profile.to_file(f"reports/ {report_name}.html")

### Tabela title_ratings

In [ ]:
profile(title_ratings, 0.1, "Title_ratings")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_principals

In [ ]:
profile(title_principals, 0.01, "Title_principals")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_episode

In [ ]:
profile(title_episode, 0.1, "Title_episode")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_crew

In [ ]:
profile(title_crew, 0.1, "Title_crew")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_basics

In [ ]:
profile(title_basics, 0.1, "Title_basics")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela title_akas

In [ ]:
profile(title_akas, 0.01, "Title_akas")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 2 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 2 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.7/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 2 missing from current font.
  font.set_text(s, 0, flags=flags)


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Tabela name_basics

In [ ]:
profile(name_basics, 0.05, "Name_basics")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]